In [ ]:
import sys
sys.path.append("../src")

import mechanism_with_policy_graph
import mechanism
import map_processor
import os
import joblib
import numpy as np
import matplotlib.pyplot as plt
import math

def cp_n_split(n_subgraph_x_nodes):
    return math.ceil(n_x_lattice/n_subgraph_x_nodes)

def perturb(mec, mp, epsilon=1, sample_num=200, sample_states=None, fix=None):
    if sample_states is None:
        sample_states = mp.possible_states
    
    set_of_connected_states = mp.make_set_of_connected_states(mp.possible_states, mp.graph_mat)
    mec.policy_mat = mp.graph_mat
    
    if fix is not None:
        true_states = [fix] * sample_num
    else:
        true_states = [np.random.choice(sample_states) for _ in range(sample_num)]
    perturbed_states = []
    
    for true_state in true_states:
        true_coord = mp.state_to_coord(true_state)
        
        connected_states_of_true_state = mp.connected_states(true_state, set_of_connected_states)
        connected_coords_of_true_state = mp.states_to_coords(connected_states_of_true_state)
        
        mec.load(connected_coords_of_true_state, connected_states_of_true_state)
        mec.build_distribution(epsilon)
        
        perturbed_coord = mec.perturb(true_coord)
        perturbed_state = mp.find_nearest_state(perturbed_coord)
        mapped_perturbed_coord = mp.state_to_coord(perturbed_state)
        
        perturbed_states.append(perturbed_state)
    
    return true_states, perturbed_states, mp

def perturb_by_epsilons(mec, mp, epsilons, sample_states=None, fix=None, sample_num=200):
    results = []
    for epsilon in epsilons:
        print("epsilon:", epsilon)
        results.append(perturb(mec, mp, epsilon, sample_states=sample_states, fix=fix, sample_num=sample_num))
    return results
    
def E_eu(result):
    true_states, perturbed_states, mp = result
    
    true_coords = mp.states_to_coords(true_states)
    perturbed_coords = mp.states_to_coords(perturbed_states)
    
    return np.average(np.linalg.norm(true_coords - perturbed_coords, axis=1) / 1000) * mp.lattice_length

def E_r(result, mp):
    true_states, perturbed_states, _ = result
    
    is_same_areas = [mp.is_same_area(true_state, perturbed_state) for true_state, perturbed_state in zip(true_states, perturbed_states)]
    
    return 1 - np.sum(is_same_areas)/len(is_same_areas)

def E_poi(result, mp):
    true_states, perturbed_states, _ = result
    
    assert all([mp.is_in(true_state) for true_state in true_states])
    
    is_same_cats = [mp.is_in(perturbed_state) for perturbed_state in perturbed_states]
    
    return 1 - np.sum(is_same_cats)/len(is_same_cats)
    
def plot_same(epsilons, pim_sames, lm_sames, filename="temp"):
    plt.rcParams["font.size"] = 20
    plt.plot(epsilons, pim_sames, label="P-PIM")
    plt.plot(epsilons, lm_sames, label="P-LM")
    plt.legend()
    plt.xlabel("$\epsilon$")
    plt.ylabel("same probability")
    plt.savefig(imgdir + filename + "_same.eps", bbox_inches='tight', pad_inches=0,  transparent=False)
    plt.show()
    
def plot(pim_results, lm_results, filename="temp"):
    imgdir = ""
    
    for pim, lm, graph_name in zip(pim_results, lm_results, graph_names):
        print(graph_name)
        plt.rcParams["font.size"] = 20
        plt.plot(epsilons, pim, label="P-PIM", marker="o")
        plt.plot(epsilons, lm, label="P-LM", linestyle="dotted", marker="x")
        plt.legend(fontsize=12)
        plt.xlabel("$\epsilon$")
        plt.ylabel("$E_{eu}$")
        plt.savefig(imgdir + filename + graph_name + ".eps", bbox_inches='tight', pad_inches=0,  transparent=False)
        plt.show()
    
    
def plot_bar(results, y_name, filename="temp"):
    k9, k16, k25, poi = results
    
    x = np.arange(len(epsilons))
    width = 0.2

    fig, ax = plt.subplots()
    rects1 = ax.bar(x - width*3/2, k9, width, label='K9')
    rects2 = ax.bar(x - width/2, k16, width, label='K16')
    rects3 = ax.bar(x + width/2, k25, width, label='K25')
    rects4 = ax.bar(x + width*3/2, poi, width, label='POI')
    ax.set_xticks(x)
    
    plt.rcParams["font.size"] = 20
    plt.legend(loc = "best", fontsize=8, ncol=2)
    ax.set_ylabel('$E_{}$'.format("{" + y_name + "}"))
    ax.set_xticklabels(epsilons)
    plt.xlabel("$\epsilon$")

    fig.tight_layout()
    plt.savefig(filename + ".eps", bbox_inches='tight', pad_inches=0,  transparent=False)
    plt.show()

In [ ]:
n_x_lattice = 50
sample_num = 2000

placeID = 3
min_lon, max_lon, min_lat, max_lat = 139.6, 139.75, 35.679, 35.8

epsilons = [0.3, 0.5, 1, 2]

list_n_subgraph_x_nodes = [3,4,5]
cat_n_subgraph_x_nodes = 6
graph_names = [f"G_k{num ** 2}" for num in list_n_subgraph_x_nodes]
graph_names.append(f"G_poi_k{cat_n_subgraph_x_nodes ** 2}")

data_dir = os.path.join("/", "data", "takagi", "globefish")
results_dir = os.path.join(data_dir, "results")

# Make Map

In [ ]:
map_processors = []

for n_subgraph_x_nodes in list_n_subgraph_x_nodes:
    n_split = cp_n_split(n_subgraph_x_nodes)

    mp = map_processor.MapProcessor(n_x_lattice)
    mp.make_map_from_latlon(min_lon, max_lon, min_lat, max_lat)
    mp.make_graph_from_area(n_split=n_split, r=float("inf"))
    mp.plot_map()
    
    map_processors.append(mp)
    
n_split = cp_n_split(cat_n_subgraph_x_nodes)
cat_mp = map_processor.MapProcessor(n_x_lattice)
cat_mp.make_map_from_latlon(min_lon, max_lon, min_lat, max_lat)
cat_mp.make_graph_from_category_and_area(placeID=placeID, n_split=n_split)
cat_mp.plot_map()
map_processors.append(cat_mp)

# Mechanisms

In [ ]:
pim_with_pg = mechanism_with_policy_graph.PlanarIsotropicMechanismWithPolicyGraph()
lm_with_pg = mechanism_with_policy_graph.LaplaceMechanismWithPolicyGraph()

pim_results = {graph_name:None for graph_name in graph_names}
pim_results_poi = {graph_name:None for graph_name in graph_names}
pim_results_part = {graph_name:None for graph_name in graph_names}
lm_results = {graph_name:None for graph_name in graph_names}

# Perturbation

In [ ]:
for mp, graph_name in zip(map_processors, graph_names):
    np.random.seed(0)
    pim_results[graph_name] = perturb_by_epsilons(pim_with_pg, mp, epsilons, sample_num=sample_num)

    np.random.seed(0)
    lm_results[graph_name] = perturb_by_epsilons(lm_with_pg, mp, epsilons, sample_num=sample_num)

In [ ]:
sample_states = map_processors[3].possible_states

for mp, graph_name in zip(map_processors, graph_names):
    print(graph_name)
    np.random.seed(0)
    pim_results_poi[graph_name] = perturb_by_epsilons(pim_with_pg, mp, epsilons, sample_states=sample_states, sample_num=sample_num)

# Accumulation

In [ ]:
target_map_processor = map_processors[3]
k9_pim_e_poi = [E_poi(pim_results_poi["G_k9"][i], target_map_processor) for i in range(len(epsilons))]
k16_pim_e_poi = [E_poi(pim_results_poi["G_k16"][i], target_map_processor) for i in range(len(epsilons))]
k25_pim_e_poi = [E_poi(pim_results_poi["G_k25"][i], target_map_processor) for i in range(len(epsilons))]
poi_pim_e_poi = [E_poi(pim_results_poi[f"G_poi_k{36}"][i], target_map_processor) for i in range(len(epsilons))]
plot_bar([k9_pim_e_poi, k16_pim_e_poi, k25_pim_e_poi, poi_pim_e_poi], y_name="POI", filename=f"poi")

In [ ]:
k9_pim_e_eus = [E_eu(pim_results["G_k9"][i]) for i in range(len(epsilons))]
k16_pim_e_eus = [E_eu(pim_results["G_k16"][i]) for i in range(len(epsilons))]
k25_pim_e_eus = [E_eu(pim_results["G_k25"][i]) for i in range(len(epsilons))]
poi_pim_e_eus = [E_eu(pim_results[f"G_poi_k{cat_n_subgraph_x_nodes ** 2}"][i]) for i in range(len(epsilons))]
plot_bar([k9_pim_e_eus, k16_pim_e_eus, k25_pim_e_eus, poi_pim_e_eus], y_name="eu", filename="eu")

In [ ]:
target_map_processor = map_processors[2]
k9_pim_e_rs = [E_r(pim_results["G_k9"][i], target_map_processor) for i in range(len(epsilons))]
k16_pim_e_rs = [E_r(pim_results["G_k16"][i], target_map_processor) for i in range(len(epsilons))]
k25_pim_e_rs = [E_r(pim_results["G_k25"][i], target_map_processor) for i in range(len(epsilons))]
poi_pim_e_rs = [E_r(pim_results[f"G_poi_k{cat_n_subgraph_x_nodes ** 2}"][i], target_map_processor) for i in range(len(epsilons))]
plot_bar([k9_pim_e_rs, k16_pim_e_rs, k25_pim_e_rs, poi_pim_e_rs], y_name="r", filename=f"r")

In [ ]:
k9_pim_e_eu = [E_eu(pim_results["G_k9"][i]) for i in range(len(epsilons))]
k16_pim_e_eu = [E_eu(pim_results["G_k16"][i]) for i in range(len(epsilons))]
k25_pim_e_eu = [E_eu(pim_results["G_k25"][i]) for i in range(len(epsilons))]
poi_pim_e_eu = [E_eu(pim_results[f"G_poi_k{cat_n_subgraph_x_nodes ** 2}"][i]) for i in range(len(epsilons))]

k9_lm_e_eu = [E_eu(lm_results["G_k9"][i]) for i in range(len(epsilons))]
k16_lm_e_eu = [E_eu(lm_results["G_k16"][i]) for i in range(len(epsilons))]
k25_lm_e_eu = [E_eu(lm_results["G_k25"][i]) for i in range(len(epsilons))]
poi_lm_e_eu = [E_eu(lm_results[f"G_poi_k{cat_n_subgraph_x_nodes ** 2}"][i]) for i in range(len(epsilons))]

plot([k9_pim_e_eu, k16_pim_e_eu, k25_pim_e_eu, poi_pim_e_eu], [k9_lm_e_eu, k16_lm_e_eu, k25_lm_e_eu, poi_lm_e_eu])

# repair graph

In [ ]:
n_x_lattice = 50
r = 700
epsilon = 1

placeID = 3
min_lon, max_lon, min_lat, max_lat = 139.6, 139.75, 35.68, 35.8

list_n_subgraph_x_nodes = [5, 6, 10, 15, 25]
mps = []

for n_subgraph_x_nodes in list_n_subgraph_x_nodes:
    mp = map_processor.MapProcessor(n_x_lattice)
    n_split = mp.cp_n_split(n_subgraph_x_nodes)
    mp.make_map_from_latlon(min_lon, max_lon, min_lat, max_lat)
    mp.make_area(n_split)
    mp.make_graph_from_category_and_distance(placeID=placeID, r=r)
    mps.append(mp)

In [ ]:
import time

pim_with_pg = mechanism_with_policy_graph.PlanarIsotropicMechanismWithPolicyGraph()

np.random.seed(0)
results = []
for i, mp in enumerate(mps):
    times = []
    distances = []
    for _ in range(200):
        true_state = np.random.choice(mp.possible_states)
        area_state = mp.state_to_area_state(true_state)

        states = [state for state in mp.areas[area_state] if state in mp.possible_states]
        
        start_time = time.time()
        repaired_graph_mat = map_processor.repair_graph(mp, states)
        end_time = time.time()
        passed_time = end_time - start_time
        print("time is:", passed_time)
        coords = mp.states_to_coords(states)
        
        pim_with_pg.load(coords, states)
        pim_with_pg.policy_mat = repaired_graph_mat
        
        pim_with_pg.build_distribution(epsilon)
        
        temps = []
        for _ in range(200):
            sampled = np.random.choice(states)
            
            true_coord = mp.state_to_coord(sampled)
            perturbed_coord = pim_with_pg.perturb(true_coord)
            perturbed_state = mp.find_nearest_state(perturbed_coord)
            mapped_perturbed_coord = mp.state_to_coord(perturbed_state)

            distance = np.linalg.norm(mapped_perturbed_coord - true_coord) * mp.lattice_length
            temps.append(distance)

        print("distance is:", np.average(temps))
        distances.append(np.average(temps))
        times.append(passed_time)
    results.append((times, distances))

In [ ]:
lengths = [(n_subgraph_x_node * mp.lattice_length / 1000) ** 2 for n_subgraph_x_node in list_n_subgraph_x_nodes]

distances = [np.average(result[1]) / 1000 for result in results]
times = [np.average(result[0]) for result in results]

plt.plot(lengths, distances, marker="o")
plt.rcParams["font.size"] = 20
plt.xlabel("area of a constraint domain (km$^2$)")
plt.ylabel("$E_{eu}$")
plt.savefig("repair_graph_domain_eeu.eps", bbox_inches='tight', pad_inches=0)
plt.show()

plt.plot(lengths, times, marker="o")
plt.rcParams["font.size"] = 20
plt.xlabel("area of a constraint domain (km$^2$)")
plt.ylabel("time (second)")
plt.savefig("repair_graph_domain_time.eps", bbox_inches='tight', pad_inches=0)
plt.show()

# Trace data

In [ ]:
import sys
sys.path.append("../src")
import trajectory_processor
import map_processor
import mechanism_with_policy_graph
import os
import numpy as np
import math

def perturb_trajectory(traj, traj_processor, epsilon, mec, iter_num=1, initial_constraint_domain_tp=None):
    reports = {"true_trajectory": traj[:100], "perturbed_trajectories": []}
    
    mec.policy_mat = traj_processor.graph_mat
    
    for f in range(iter_num):
        
        print("\n" + str(f))
        
        if initial_constraint_domain_tp:
            initial_constraint_domain = initial_constraint_domain_tp.areas[initial_constraint_domain_tp.state_to_area_state(traj[0])]
            prob = 1/len(initial_constraint_domain)

            prior_distribution = np.zeros(len(traj_processor.transition_mat[0]))
            prior_distribution[np.array(initial_constraint_domain)] = prob
        else:
            prior_distribution = np.zeros(len(traj_processor.transition_mat[0]))
            prior_distribution[traj[0]] = 1
        
        perturbed_trajectory = np.zeros(len(traj[:100]))

        for i, true_state in enumerate(traj[:100]):
            print(i, end="\r")

            pos_dist = traj_processor.compute_posterior_distribution(prior_distribution)
            state_nos = traj_processor.compute_possible_set(pos_dist, delta=0)
            
            set_of_connected_states = traj_processor.make_set_of_connected_states(state_nos, traj_processor.graph_mat)
            connected_states_of_true_state = traj_processor.connected_states(true_state, set_of_connected_states)
            connected_coords_of_true_state = traj_processor.states_to_coords(connected_states_of_true_state)
            
            true_coord = traj_processor.state_to_coord(true_state)
            
            mec.load(connected_coords_of_true_state, connected_states_of_true_state)
            mec.build_distribution(epsilon)
            
            perturbed_coord = mec.perturb(true_coord)
            perturbed_state = traj_processor.find_nearest_state(perturbed_coord)
            perturbed_trajectory[i] = perturbed_state
            
            prior_distribution = mec.inference(pos_dist, perturbed_coord)
            
        reports["perturbed_trajectories"].append(perturbed_trajectory)
        
    return reports

def perturb_trajectories(trajs, traj_processor, epsilon, mec, iter_num=1, initial_constraint_domain=None):
    reports = []
    for traj in trajs:
        reports.append(perturb_trajectory(traj, traj_processor, epsilon, mec, iter_num=iter_num, initial_constraint_domain_tp=initial_constraint_domain))
    return reports

In [ ]:
pim_with_pg = mechanism_with_policy_graph.PlanarIsotropicMechanismWithPolicyGraph()
lm_with_pg = mechanism_with_policy_graph.LaplaceMechanismWithPolicyGraph()

## Geolife

In [ ]:
geolife_datadir = "/data/takagi/globefish/Geolife Trajectories 1.3/Data"
users = os.listdir(geolife_datadir)

def extract_latlon(record):
    record = record.split(",")
    return [float(record[0]), float(record[1])]

trajs = []
for user in users:
    datadir = os.path.join(geolife_datadir, user)
    traj_dirs = os.listdir(os.path.join(datadir, "Trajectory"))
    
    trajs_for_each_user = []
    for traj_dir in traj_dirs:
        traj_dir = os.path.join(datadir, "Trajectory", traj_dir)
        with open(traj_dir) as f:
            traj = [extract_latlon(record) for record in f.readlines()[6:]]
            trajs.append(traj)

In [ ]:
tp = trajectory_processor.TrajectoryProcessor(n_x_lattice)
tp.make_map_from_latlon(max_lat=39.988695, max_lon=116.467438, min_lat=39.831741, min_lon=116.273804)
n_split = tp.cp_n_split(5)
tp.make_graph_from_area(n_split=n_split, r=float("inf"))
state_trajs = tp.trajs_to_state_trajs(trajs)
tp.make_transmat_from_state_trajs(state_trajs)

In [ ]:
list_n_subgraph_x_nodes = [3, 4, 5]

tps = []

for n_subgraph_x_nodes in list_n_subgraph_x_nodes:
    tp_ = trajectory_processor.TrajectoryProcessor(n_x_lattice)
    n_split = tp_.cp_n_split(n_subgraph_x_nodes)
    tp_.make_map_from_latlon(max_lat=39.988695, max_lon=116.467438, min_lat=39.831741, min_lon=116.273804)
    tp_.make_graph_from_area(n_split=n_split, r=float("inf"))
    tp_.transition_mat = tp.transition_mat
    
    tps.append(tp_)

In [ ]:
def smoothing(result, win=10):
    return [np.average(result[max(i-win*2, 0):i+win*2]) for i in range(len(result))]

def distance(true_state_traj, perturbed_state_trajs, tp):
    distances = []
    true_coords = tp.states_to_coords(true_state_traj)
    for perturbed_state_traj in perturbed_state_trajs:
        perturbed_coords = tp.states_to_coords(perturbed_state_traj)[:100]
        distances.append(np.linalg.norm(true_coords - perturbed_coords, axis=1) * tp.lattice_length)
    return np.average(distances, axis=0)

def ave_distance(reports, tp):
    distances = []
    for report in reports:
        distances.append(distance(report["true_trajectory"], report["perturbed_trajectories"], tp))
    return np.average(distances, axis=0) / 1000

def ave_ave_distance(reports, tp):
    distances = []
    for report in reports:
        distances.append(distance(report["true_trajectory"], report["perturbed_trajectories"], tp))
    return np.average(distances)

In [ ]:
import joblib
epsilons = [0.3, 0.5, 1, 2]

for epsilon in epsilons:
    for i, tp in enumerate(tps):

        np.random.seed(0)
        pim_reports = perturb_trajectories(state_trajs[:20], tp, epsilon, pim_with_pg, iter_num=20)
        lm_reports = perturb_trajectories(state_trajs[:20], tp, epsilon, lm_with_pg, iter_num=20)

        joblib.dump(filename=f"epsilon_{epsilon}_{i}.jbl", value=[pim_reports, lm_reports])

In [ ]:
epsilon = 2
k9_pim_reports, k9_lm_reports = joblib.load(filename=f"epsilon_{epsilon}_0.jbl")
k16_pim_reports, k16_lm_reports = joblib.load(filename=f"epsilon_{epsilon}_1.jbl")
k25_pim_reports, k25_lm_reports = joblib.load(filename=f"epsilon_{epsilon}_2.jbl")

k9_2_pim_distance = ave_ave_distance(k9_pim_reports, tps[0])
k16_2_pim_distance = ave_ave_distance(k16_pim_reports, tps[1])
k25_2_pim_distance = ave_ave_distance(k25_pim_reports, tps[2])

epsilon = 0.3
k9_pim_reports, k9_lm_reports = joblib.load(filename=f"epsilon_{epsilon}_0.jbl")
k16_pim_reports, k16_lm_reports = joblib.load(filename=f"epsilon_{epsilon}_1.jbl")
k25_pim_reports, k25_lm_reports = joblib.load(filename=f"epsilon_{epsilon}_2.jbl")

k9_03_pim_distance = ave_ave_distance(k9_pim_reports, tps[0])
k16_03_pim_distance = ave_ave_distance(k16_pim_reports, tps[1])
k25_03_pim_distance = ave_ave_distance(k25_pim_reports, tps[2])

epsilon = 0.5
k9_pim_reports, k9_lm_reports = joblib.load(filename=f"epsilon_{epsilon}_0.jbl")
k16_pim_reports, k16_lm_reports = joblib.load(filename=f"epsilon_{epsilon}_1.jbl")
k25_pim_reports, k25_lm_reports = joblib.load(filename=f"epsilon_{epsilon}_2.jbl")

k9_05_pim_distance = ave_ave_distance(k9_pim_reports, tps[0])
k16_05_pim_distance = ave_ave_distance(k16_pim_reports, tps[1])
k25_05_pim_distance = ave_ave_distance(k25_pim_reports, tps[2])

epsilon = 1
k9_pim_reports, k9_lm_reports = joblib.load(filename=f"epsilon_{epsilon}_0.jbl")
k16_pim_reports, k16_lm_reports = joblib.load(filename=f"epsilon_{epsilon}_1.jbl")
k25_pim_reports, k25_lm_reports = joblib.load(filename=f"epsilon_{epsilon}_2.jbl")

k9_1_pim_distance = ave_ave_distance(k9_pim_reports, tps[0])
k16_1_pim_distance = ave_ave_distance(k16_pim_reports, tps[1])
k25_1_pim_distance = ave_ave_distance(k25_pim_reports, tps[2])

k9 = [k9_03_pim_distance, k9_05_pim_distance, k9_1_pim_distance, k9_2_pim_distance]
k16 = [k16_03_pim_distance, k16_05_pim_distance, k16_1_pim_distance, k16_2_pim_distance]
k25 = [k25_03_pim_distance, k25_05_pim_distance, k25_1_pim_distance, k25_2_pim_distance]

In [ ]:
def plot_bar(results, y_name, filename="temp"):
    k9, k16, k25 = results
    
    x = np.arange(len(epsilons))
    width = 0.2

    fig, ax = plt.subplots()
    rects1 = ax.bar(x - width, np.array(k9) / 1000, width, label='K9')
    rects2 = ax.bar(x, np.array(k16) / 1000, width, label='K16')
    rects3 = ax.bar(x + width, np.array(k25) /1000, width, label='K25')
    ax.set_xticks(x)
    
    plt.rcParams["font.size"] = 20
    plt.legend(loc = "best", fontsize=10)
    ax.set_ylabel('average of $E_{}$'.format("{" + y_name + "}"))
    ax.set_xticklabels(epsilons)
    plt.xlabel("$\epsilon$")

    fig.tight_layout()
    plt.savefig(filename + ".eps", bbox_inches='tight', pad_inches=0,  transparent=False)
    plt.show()
    
plot_bar([k9, k16, k25], "eu")

In [ ]:
k9_pim_distances = ave_distance(k9_pim_reports, tp)
k16_pim_distances = ave_distance(k16_pim_reports, tp)
k25_pim_distances = ave_distance(k25_pim_reports, tp)

plt.rcParams["font.size"] = 20
plt.ylabel("$E_{eu}$")
plt.xlabel("timestamp")
plt.plot(smoothing(k9_pim_distances), label="$G_{k9}$", linestyle="dotted")
plt.plot(smoothing(k16_pim_distances), label="$G_{k16}$", linestyle="dashed")
plt.plot(smoothing(k25_pim_distances), label="$G_{k25}$")
plt.legend(fontsize=10, loc="upper right", bbox_to_anchor=(1,0.9))
plt.savefig("geolife.eps", bbox_inches='tight', pad_inches=0 )

In [ ]:
pim_distances = ave_distance(k9_pim_reports, tp)
lm_distances = ave_distance(k9_lm_reports, tp)

plt.rcParams["font.size"] = 20
plt.ylabel("$E_{eu}$")
plt.xlabel("timestamp")
plt.plot(smoothing(pim_distances), label="P-PIM")
plt.plot(smoothing(lm_distances), label="P-LM", linestyle="dotted")
plt.legend(fontsize=10)
plt.savefig("9k.eps", bbox_inches='tight', pad_inches=0 )

In [ ]:
pim_distances = ave_distance(k16_pim_reports, tp)
lm_distances = ave_distance(k16_lm_reports, tp)

plt.rcParams["font.size"] = 20
plt.ylabel("$E_{eu}$")
plt.xlabel("timestamp")
plt.plot(smoothing(pim_distances), label="P-PIM")
plt.plot(smoothing(lm_distances), label="P-LM", linestyle="dotted")
plt.legend(fontsize=10)
plt.savefig("16k.eps", bbox_inches='tight', pad_inches=0 )

In [ ]:
pim_distances = ave_distance(k25_pim_reports, tp)
lm_distances = ave_distance(k25_lm_reports, tp)

plt.rcParams["font.size"] = 20
plt.ylabel("$E_{eu}$")
plt.xlabel("timestamp")
plt.plot(smoothing(pim_distances), label="P-PIM")
plt.plot(smoothing(lm_distances), label="P-LM", linestyle="dotted")
plt.legend(fontsize=10)
plt.savefig("25k.eps", bbox_inches='tight', pad_inches=0 )

# Gowalla

In [ ]:
def judge_in(latlon):
    return (min_lat <= latlon[0] <= max_lat) and (min_lon <= -latlon[1] <= max_lon)

gowalla_datadir = "/data/takagi/globefish/loc-gowalla_totalCheckins.txt"
n_id = 107093
trajs = {}
with open(gowalla_datadir) as gowalla_data:
    pre_id = 0
    for loc in gowalla_data:
        splitted = loc.split("\t")
        id = int(splitted[0])
        if id not in trajs.keys():
            trajs[id] = []
        else:
            trajs[id].append((float(splitted[2]), float(splitted[3])))

In [ ]:
n_x_lattice = 50
min_lat = 34.02
max_lat = 34.18
min_lon = 118.2
max_lon = 118.4
            
in_trajs = []
for traj in trajs.values():
    in_traj = np.array(traj)[np.where([judge_in(latlon) for latlon in traj])]
    if not len(in_traj) <= 1:
        in_trajs.append(np.abs(in_traj))
        
tp = trajectory_processor.TrajectoryProcessor(n_x_lattice)
tp.make_map_from_latlon(max_lat=max_lat, max_lon=max_lon, min_lat=min_lat, min_lon=min_lon)
state_trajs = tp.trajs_to_state_trajs(in_trajs)
tp.make_transmat_from_state_trajs(state_trajs)

test_trajs = [traj for traj in state_trajs if len(traj) >= 100]

In [ ]:

def perturb_trajectory(traj, traj_processor, epsilon, mec, iter_num=1):
    reports = {"true_trajectory": traj[:100], "perturbed_trajectories": []}
    
    mec.policy_mat = traj_processor.graph_mat
    
    for f in range(iter_num):
        
        print("\n" + str(f))
        
        prior_distribution = np.zeros(len(traj_processor.transition_mat[0]))
        prior_distribution[traj[0]] = 1
        
        perturbed_trajectory = np.zeros(len(traj[:100]))

        for i, true_state in enumerate(traj[:100]):
            print(i, end="\r")

            pos_dist = traj_processor.compute_posterior_distribution(prior_distribution)
            state_nos = traj_processor.compute_possible_set(pos_dist, delta=0)
            
            set_of_connected_states = traj_processor.make_set_of_connected_states(state_nos, traj_processor.graph_mat)
            connected_states_of_true_state = traj_processor.connected_states(true_state, set_of_connected_states)
            if connected_states_of_true_state == None:
                connected_states_of_true_state = traj_processor.areas[traj_processor.state_to_area_state(true_state)]
            connected_coords_of_true_state = traj_processor.states_to_coords(connected_states_of_true_state)
            
            true_coord = traj_processor.state_to_coord(true_state)
            
            mec.load(connected_coords_of_true_state, connected_states_of_true_state)
            mec.build_distribution(epsilon)
            
            perturbed_coord = mec.perturb(true_coord)
            perturbed_state = traj_processor.find_nearest_state(perturbed_coord)
            perturbed_trajectory[i] = perturbed_state
            
            prior_distribution = mec.inference(pos_dist, perturbed_coord)
            
        reports["perturbed_trajectories"].append(perturbed_trajectory)
        
    return reports

def perturb_trajectories(trajs, traj_processor, epsilon, mec, iter_num=1):
    reports = []
    for traj in trajs:
        reports.append(perturb_trajectory(traj, traj_processor, epsilon, mec, iter_num=iter_num))
    return reports

In [ ]:
list_n_subgraph_x_nodes = [3, 4, 5]

tps = []

for n_subgraph_x_nodes in list_n_subgraph_x_nodes:
    tp_ = trajectory_processor.TrajectoryProcessor(n_x_lattice)
    n_split = tp_.cp_n_split(n_subgraph_x_nodes)
    tp_.make_map_from_latlon(max_lat=max_lat, max_lon=max_lon, min_lat=min_lat, min_lon=min_lon)
    tp_.make_graph_from_area(n_split=n_split, r=float("inf"))
    tp_.transition_mat = tp.transition_mat
    
    tps.append(tp_)

In [ ]:
epsilons = [1]
pim_with_pg = mechanism_with_policy_graph.PlanarIsotropicMechanismWithPolicyGraph()

reports = []
for epsilon in epsilons:
    for i, tp in enumerate(tps):

        np.random.seed(0)
        pim_reports = perturb_trajectories(test_trajs[:20], tp, epsilon, pim_with_pg, iter_num=20)
        reports.append(pim_reports)

In [ ]:
k9_pim_distances = ave_distance(reports[0], tp)
k16_pim_distances = ave_distance(reports[1], tp)
k25_pim_distances = ave_distance(reports[2], tp)

plt.rcParams["font.size"] = 20
plt.ylabel("$E_{eu}$")
plt.xlabel("timestamp")
plt.plot(smoothing(k9_pim_distances), label="$G_{k9}$", linestyle="dotted")
plt.plot(smoothing(k16_pim_distances), label="$G_{k16}$", linestyle="dashed")
plt.plot(smoothing(k25_pim_distances), label="$G_{k25}$")
plt.legend(fontsize=10, loc="upper right", bbox_to_anchor=(1,0.9))
plt.savefig("gowalla.eps", bbox_inches='tight', pad_inches=0 )